excel設計

In [1]:
# -*- coding: utf-8 -*-
"""
@author: Gemini (Final Corrected Version 5.7)
@date: 2025-10-10
@description: 
    此腳本為「交屋稅費分算計算機」的功能更新版本。
    v5.7 更新日誌:
    1.  [功能] 新增【檔案作者屬性】，將 Excel 檔案的「作者」資訊正式設定為「陳定康」。
             可從 [檔案] > [資訊] 中查看。
    2.  [功能] 在工作表左下角新增【設計者頁尾】，讓使用者能直接看到作者資訊。
    v5.6 更新日誌:
    1.  [功能] 新增【日期邏輯驗證】，強制要求「交屋日」必須大於或等於「過戶日」。
    2.  [功能] 新增【工作表保護】，鎖定所有非輸入欄位，防止公式或備註被意外修改。
    3.  [調整] 將所有儲存格備註作者更新為「陳定康」。
"""
import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side, Protection
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.comments import Comment


def apply_styles_to_range(ws, cell_range, styles):
    """輔助函式：對指定的儲存格範圍一次性套用多種樣式。"""
    valid_styles = ['font', 'fill', 'alignment', 'border', 'number_format', 'protection']
    rows = ws[cell_range]
    if not isinstance(rows, tuple):
        rows = ((rows,),)
    for row in rows:
        for cell in row:
            for style_name, style_obj in styles.items():
                if style_name in valid_styles and style_obj is not None:
                    setattr(cell, style_name, style_obj)


def create_final_workbook_v5_7(filename="交屋稅費計算機.xlsx"):
    """建立、格式化並儲存包含複雜公式與樣式的稅費計算機 Excel 檔案。"""
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "專業稅費分算計算機"
    
    # --- 【新功能】設定檔案的核心屬性 ---
    wb.properties.creator = "陳定康"
    wb.properties.title = "不動產買賣稅費分算計算機"
    wb.properties.description = "此工具用於精確計算不動產買賣過程中，買賣雙方應分攤之地價稅、房屋稅及其他相關費用。"

    # --- 樣式定義 ---
    unlocked = Protection(locked=False)
    locked = Protection(locked=True)
    
    fill_yellow_input = PatternFill(
        start_color='FFFFE0', end_color='FFFFE0', fill_type='solid')
    fill_blue_input = PatternFill(
        start_color='EAF1FF', end_color='EAF1FF', fill_type='solid')
    fill_title_green = PatternFill(
        start_color='D7E9D1', end_color='D7E9D1', fill_type='solid')
    fill_header_grey = PatternFill(
        start_color='E7E6E6', end_color='E7E6E6', fill_type='solid')
    fill_result_orange = PatternFill(
        start_color='FFF2E6', end_color='FFF2E6', fill_type='solid')
    fill_period_land = PatternFill(
        start_color='E2EFDA', end_color='E2EFDA', fill_type='solid')
    fill_period_house = PatternFill(
        start_color='DEEBF7', end_color='DEEBF7', fill_type='solid')
    font_title = Font(name='微軟正黑體', size=18, bold=True)
    font_section_header = Font(name='微軟正黑體', size=14, bold=True)
    font_body = Font(name='微軟正黑體', size=12)
    font_body_bold = Font(name='微軟正黑體', size=12, bold=True)
    font_red_bold = Font(name='微軟正黑體', size=12, bold=True, color="C00000")
    font_final_result = Font(
        name='微軟正黑體', size=16, bold=True, color="C00000")
    font_note = Font(name='微軟正黑體', size=11, italic=True)
    font_footer = Font(name='微軟正黑體', size=10, color="808080")
    align_center = Alignment(
        horizontal='center', vertical='center', wrap_text=True)
    align_left = Alignment(
        horizontal='left', vertical='center', wrap_text=True)
    align_right = Alignment(
        horizontal='right', vertical='center', wrap_text=True)
    thin_border = Border(left=Side(style='thin'), right=Side(
        style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    medium_border = Border(left=Side(style='medium'), right=Side(
        style='medium'), top=Side(style='medium'), bottom=Side(style='medium'))

    # --- 欄寬與列高設定 ---
    for col, width in {'A': 25, 'B': 15, 'C': 15, 'D': 12, 'E': 15, 'F': 45, 'G': 15, 'I': 80}.items():
        ws.column_dimensions[col].width = width
    for i in range(1, 40):
        ws.row_dimensions[i].height = 25
    for row, height in {1: 40, 2: 30, 3: 30, 4: 30, 7: 30, 12: 35, 13: 35, 14: 35, 15: 35, 16: 45, 17: 45, 18: 30, 26: 35, 28: 35, 29: 45}.items():
        ws.row_dimensions[row].height = height
    ws.row_dimensions[3].height = 110
    ws.row_dimensions[9].height = 110
    
    # --- 輔助計算區 ---
    ws.column_dimensions['AA'].hidden = True
    ws.column_dimensions['AB'].hidden = True
    ws['AA1'] = '=IFERROR(IF(LEN(B2)=7, DATE(VALUE(LEFT(B2,3))+1911, VALUE(MID(B2,4,2)), VALUE(RIGHT(B2,2))), IF(LEN(B2)=6, DATE(VALUE(LEFT(B2,2))+1911, VALUE(MID(B2,3,2)), VALUE(RIGHT(B2,2))), "")), "")'
    ws['AA2'] = '=IFERROR(IF(LEN(B3)=7, DATE(VALUE(LEFT(B3,3))+1911, VALUE(MID(B3,4,2)), VALUE(RIGHT(B3,2))), IF(LEN(B3)=6, DATE(VALUE(LEFT(B3,2))+1911, VALUE(MID(B3,3,2)), VALUE(RIGHT(B3,2))), "")), "")'
    ws['AA3'] = '=IFERROR(IF(C7<>"", (LEFT(C7, FIND("/", C7)-1) / MID(C7, FIND("/", C7)+1, LEN(C7))) * 100, B7), B7)'
    ws['AA4'] = '=IF(B4="是", 1, 0)'
    ws['AA5'] = '=IF(AA2="","",AND(DAY(EOMONTH(AA2,0))=DAY(AA2), B4="否"))'
    ws['AA6'] = '=IFERROR(IF(LEN(B22)=7, DATE(VALUE(LEFT(B22,3))+1911, VALUE(MID(B22,4,2)), VALUE(RIGHT(B22,2))), IF(LEN(B22)=6, DATE(VALUE(LEFT(B22,2))+1911, VALUE(MID(B22,3,2)), VALUE(RIGHT(B22,2))), "")), "")'
    ws['AA7'] = '=IFERROR(IF(LEN(B23)=7, DATE(VALUE(LEFT(B23,3))+1911, VALUE(MID(B23,4,2)), VALUE(RIGHT(B23,2))), IF(LEN(B23)=6, DATE(VALUE(LEFT(B23,2))+1911, VALUE(MID(B23,3,2)), VALUE(RIGHT(B23,2))), "")), "")'

    # --- 核心公式定義 ---
    calc_formulas = {
        'AB1': '=IF(B12<>"", B12, IFERROR(ROUND(B5*B6*(AA3/100)*VLOOKUP(B8,稅率表!$D$2:$E$3,2,FALSE),0),""))',
        'AB2': '=IF(B13<>"", B13, IFERROR(ROUND(B9*VLOOKUP(B10, 稅率表!$A$2:$B$14, 2, FALSE),0), ""))',
        'AB3': '=IF(AA5, 12, IFERROR(DATE(YEAR(AA2),12,31)-DATE(YEAR(AA2),1,1)+1, ""))',
        'AB4': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0),7,1), "")',
        'AB5': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0)+1,6,30), "")',
        'AB6': '=IF(AA5, 12, IFERROR(AB5-AB4+1, ""))',
        'AB7': '=IFERROR(EOMONTH(DATE(YEAR(AB5),2,1),0), "")',
        'AB8': '=IFERROR(IF(AA1>DATE(YEAR(AA2),8,31), "賣方", "買方"), "")',
        'AB9': '=IFERROR(IF(AA1>AB7, "賣方", "買方"), "")',
        'AB10': '=IFERROR(IF(AA5, MONTH(AA2), AA2-DATE(YEAR(AA2),1,1)+1-AA4),"")',
        'AB11': '=IFERROR(IF(AA5, 12-MONTH(AA2), DATE(YEAR(AA2),12,31)-AA2+AA4),"")',
        'AB12': '=IFERROR(IF(AA5, IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AA2-AB4+1-AA4),"")',
        'AB13': '=IFERROR(IF(AA5, 12 - IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AB5-AA2+AA4),"")',
        'AB21': '=IFERROR(IF(AND(B20>0, AA6>=AA2), ROUND(B20 * (AA6-AA2+AA4) / DAY(EOMONTH(AA2,0)), 0), 0), 0)',
        'AB22': '=IFERROR(IF(AND(B21>0, AA7>=AA2), ROUND(B21 * (AA7-AA2+AA4) / DAY(EOMONTH(AA2,0)), 0), 0), 0)',
        'AB23': '=AB21+AB22',
        'AB25': '=IFERROR(IF(AB8="買方", E16, 0) + IF(AB9="買方", E17, 0), 0)',
        'AB26': '=IFERROR(IF(AB8="賣方", E16, 0) + IF(AB9="賣方", E17, 0) + AB23, 0)',
        'B16': '=IF(AB10<>"", AB10 & IF(AA5, " 月", " 天"), "")',
        'C16': '=IF(AB11<>"", AB11 & IF(AA5, " 月", " 天"), "")',
        'D16': '=AB1',
        'E16': '=IFERROR(IF(AB8="賣方", ROUND(D16*AB11/AB3,0), ROUND(D16*AB10/AB3,0)),0)',
        'F16': '=IF(E16=0, "無需找補", "納稅人為【" & AB8 & "】，" & IF(AB8="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。計算式：" & TEXT(D16, "#,##0") & " × (" & IF(AB8="賣方", AB11, AB10) & " / " & AB3 & ") = " & TEXT(E16, "#,##0"))',
        'B17': '=IF(AB12<>"", AB12 & IF(AA5, " 月", " 天"), "")',
        'C17': '=IF(AB13<>"", AB13 & IF(AA5, " 月", " 天"), "")',
        'D17': '=AB2',
        'E17': '=IFERROR(IF(AB9="賣方", ROUND(D17*AB13/AB6,0), ROUND(D17*AB12/AB6,0)),0)',
        'F17': '=IF(E17=0, "無需找補", "納稅人為【" & AB9 & "】，" & IF(AB9="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。計算式：" & TEXT(D17, "#,##0") & " × (" & IF(AB9="賣方", AB13, AB12) & " / " & AB6 & ") = " & TEXT(E17, "#,##0"))',
        'D24': '=AB23',
        'B28': '=AB25', 'C28': '=AB26',
        'B29': '=IF(C28>B28, "【買方】應支付給【賣方】 NT$ " & TEXT(C28-B28, "#,##0"), IF(B28>C28, "【賣方】應支付給【買方】 NT$ " & TEXT(B28-C28, "#,##0"), "雙方無需找補"))',
        'I2': '=IF(AA2="","", "一、地價稅")',
        'I3': '=IF(AA2="","", CONCATENATE("● 開徵期間：每年 11 月 1 日至 11 月 30 日止。", CHAR(10), "● 納稅義務基準日：每年 8 月 31 日。", CHAR(10), "● 本案課稅年度：民國 ", YEAR(AA2)-1911, " 年 1 月 1 日至 ", YEAR(AA2)-1911, " 年 12 月 31 日。", CHAR(10), "● 繳納義務人：分算後，民國 ", YEAR(AA2)-1911, " 年度開徵之地價稅單將由【", AB8, "】負責繳納。"))',
        'I8': '=IF(AA2="","", "二、房屋稅")',
        'I9': '=IF(AA2="","", CONCATENATE("● 開徵期間：每年 5 月 1 日至 5 月 31 日止。", CHAR(10), "● 納稅義務基準日：每年 2 月之末日。", CHAR(10), "● 本案課稅期間：自民國 ", YEAR(AB4)-1911, " 年 7 月 1 日起至 ", YEAR(AB5)-1911, " 年 6 月 30 日止。", CHAR(10), "● 繳納義務人：分算後，民國 ", YEAR(AB5)-1911, " 年度開徵之房屋稅單將由【", AB9, "】負責繳納。"))',
    }
    for cell_ref, formula in calc_formulas.items():
        ws[cell_ref] = formula

    # --- 介面佈局與靜態文字 ---
    # ... (此處程式碼與 v5.6 完全相同，故省略以節省篇幅) ...
    ws.merge_cells('A1:F1')
    ws['A1'] = "不動產買賣稅費分算計算機"
    apply_styles_to_range(ws, 'A1', {
                        'font': font_title, 'fill': fill_title_green, 'alignment': align_center})
    ws['A2'] = "權狀登記(過戶)日期"
    ws['A3'] = "房屋點交(交屋)日期"
    ws['A4'] = "是否帶租約交屋"
    apply_styles_to_range(ws, 'A2:A4', {
                        'font': font_body, 'fill': fill_header_grey, 'alignment': align_right})
    ws['B2'] = 1141001
    ws['B3'] = 1141031
    ws['B4'] = "否"
    
    editable_cells = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'C7', 'B8', 'B9', 'B10', 
                      'B12', 'B13', 'B20', 'B21', 'B22', 'B23']
    for cell_ref in editable_cells:
        ws[cell_ref].protection = unlocked

    apply_styles_to_range(ws, 'B2:B4', {
                        'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    dv_lease = DataValidation(type="list", formula1='"是,否"')
    dv_lease.add('B4')
    ws.add_data_validation(dv_lease)
    ws['C2'] = '=IF(AA1<>"", TEXT(AA1, "民國 e 年 m 月 d 日"), "請依左側格式輸入日期")'
    ws['C3'] = '=IF(AA2<>"", TEXT(AA2, "民國 e 年 m 月 d 日"), "請依左側格式輸入日期")'
    ws.merge_cells('C2:D2')
    ws.merge_cells('C3:D3')
    ws.merge_cells('C4:D4')
    apply_styles_to_range(
        ws, 'C2:C3', {'font': font_body, 'alignment': align_left})
    ws['C4'] = '=IF(AA5, "【按月分算模式】", "【按日分算模式】") & " 交屋日歸屬：" & IF(B4="是", "買方", "賣方")'
    apply_styles_to_range(
        ws, 'C4', {'font': font_body_bold, 'alignment': align_left})
    ws.merge_cells('E2:F4')
    ws['E2'] = '=IF(AA1="","",IF(MOD(YEAR(AA1)-1911,2)=1, "【注意】\n民國 " & YEAR(AA1)-1911 & " 年為地價調整年！\n前一年稅單僅供參考。", "【資訊】\n民國 " & YEAR(AA1)-1911 & " 年非地價調整年。"))'
    apply_styles_to_range(
        ws, 'E2', {'font': font_red_bold, 'alignment': align_center})
    ws.merge_cells('A5:A10')
    ws['A5'] = "模式一：詳細資料輸入"
    apply_styles_to_range(ws, 'A5', {
                        'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    for cell_ref, label in {"E5": "申報地價 (元/m²)", "E6": "土地面積 (m²)", "E7": "權利範圍 (% 或 分數)", "E8": "地價稅率", "E9": "房屋現值 (元)", "E10": "房屋使用情境"}.items():
        ws.merge_cells(f'{cell_ref[0]}{cell_ref[1:]}:F{cell_ref[1:]}')
        ws[cell_ref] = label
        apply_styles_to_range(
            ws, cell_ref, {'font': font_body, 'alignment': align_left})
    for row_num in [5, 6, 8, 9]:
        ws.merge_cells(f'B{row_num}:D{row_num}')
    ws.merge_cells('C7:D7')
    ws.merge_cells('B10:C10')
    apply_styles_to_range(ws, 'B5:B10', {
                        'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    apply_styles_to_range(ws, 'D10', {
                        'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    ws['B5'] = 155091.2
    ws['B6'] = 449
    ws['B7'] = 1.28
    ws['B7'].number_format = '0.00"%"'
    ws['C7'] = ""
    ws['B8'] = "一般用地 (千分之十)"
    ws['B9'] = 1084300
    ws['B10'] = "自住用-全國3戶內"
    ws['D10'] = '=IFERROR(VLOOKUP(B10, 稅率表!$A$2:$B$14, 2, FALSE), "")'
    ws['D10'].number_format = '0.00%'
    ws.merge_cells('A12:A13')
    ws['A12'] = "模式二：快速稅額輸入"
    apply_styles_to_range(ws, 'A12', {
                        'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws.merge_cells('B12:D12')
    ws.merge_cells('B13:D13')
    apply_styles_to_range(ws, 'B12:B13', {
                        'font': font_body, 'fill': fill_blue_input, 'alignment': align_center})
    ws['E12'] = "年度應納地價稅 (快速)"
    ws['E13'] = "年度應納房屋稅 (快速)"
    apply_styles_to_range(
        ws, 'E12:E13', {'font': font_body, 'alignment': align_left})
    ws.merge_cells('F12:F13')
    ws['F12'] = "若填寫此區藍色欄位，將優先採用此處稅額進行計算。"
    apply_styles_to_range(
        ws, 'F12', {'font': font_note, 'alignment': align_center})
    ws.merge_cells('A14:F14')
    ws['A14'] = "（二）核心稅費計算"
    apply_styles_to_range(ws, 'A14', {
                        'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    for col_idx, header in enumerate(["項目", "賣方持有期間", "買方持有期間", "年度總稅額", "找補金額", "備註 (計算過程)"], 1):
        ws.cell(row=15, column=col_idx, value=header)
    apply_styles_to_range(ws, 'A15:F15', {
                        'font': font_body_bold, 'fill': fill_header_grey, 'alignment': align_center})
    ws['A16'] = "地價稅"
    ws['A17'] = "房屋稅"
    apply_styles_to_range(
        ws, 'A16:A17', {'font': font_body, 'alignment': align_center})
    apply_styles_to_range(
        ws, 'B16:C16', {'font': font_body, 'alignment': align_center, 'fill': fill_period_land})
    apply_styles_to_range(
        ws, 'B17:C17', {'font': font_body, 'alignment': align_center, 'fill': fill_period_house})
    apply_styles_to_range(
        ws, 'F16:F17', {'font': font_body, 'alignment': align_left, 'wrap_text': True})
    apply_styles_to_range(ws, 'D16:E17', {
                        'font': font_body, 'alignment': align_center, 'number_format': '#,##0'})
    ws.merge_cells('A19:F19')
    ws['A19'] = "（三）其他費用分算"
    apply_styles_to_range(ws, 'A19', {
                        'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws['A20'] = "管理費(月)"
    ws['A21'] = "車位清潔費(月)"
    ws['A22'] = "管理費已預繳至"
    ws['A23'] = "車位清潔費已預繳至"
    apply_styles_to_range(ws, 'A20:A23', {
                        'font': font_body, 'fill': fill_header_grey, 'alignment': align_right})
    apply_styles_to_range(ws, 'B20:B23', {
                        'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    ws['B20'] = 4430
    ws['B21'] = 1200
    ws['B22'] = 1141031
    ws['B23'] = 1141031
    ws['C22'] = '=IF(AA6<>"", TEXT(AA6, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    ws['C23'] = '=IF(AA7<>"", TEXT(AA7, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    ws.merge_cells('C20:F21')
    ws['C20'] = "請填寫月繳金額，若無則填 0。\n找補金額將依交屋日與費用預繳截止日，按當月天數比例分算。"
    apply_styles_to_range(
        ws, 'C20', {'font': font_note, 'alignment': align_left})
    ws.merge_cells('D22:F22')
    ws.merge_cells('D23:F23')
    ws.merge_cells('A24:C24')
    ws['A24'] = "其他費用找補金額"
    apply_styles_to_range(
        ws, 'A24', {'font': font_body_bold, 'alignment': align_center})
    ws.merge_cells('D24:F24')
    apply_styles_to_range(ws, 'D24', {
                        'font': font_body_bold, 'number_format': '#,##0', 'alignment': align_center})
    ws.merge_cells('A25:F25')
    ws['A25'] = '=IF(D24>0, "買方應補貼賣方 NT$ " & TEXT(D24, "#,##0") & " 元", "其他費用無需找補")'
    apply_styles_to_range(
        ws, 'A25', {'font': font_body, 'alignment': align_center})
    ws.merge_cells('A26:F26')
    ws['A26'] = "（四）最終結算"
    apply_styles_to_range(ws, 'A26', {
                        'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws.merge_cells('A27:A28')
    ws['A27'] = "應付總計"
    apply_styles_to_range(
        ws, 'A27', {'font': font_body_bold, 'alignment': align_center})
    ws['B27'] = "買方應付"
    ws['C27'] = "賣方應付"
    apply_styles_to_range(ws, 'B27:C27', {
                        'font': font_body, 'alignment': align_center, 'fill': fill_header_grey})
    apply_styles_to_range(ws, 'B28:C28', {
                        'font': font_body_bold, 'number_format': '"NT$ "#,##0', 'alignment': align_center})
    ws.merge_cells('D27:F28')
    ws.merge_cells('A29:A30')
    ws['A29'] = "最終結算結果"
    apply_styles_to_range(ws, 'A29', {
        'font': font_section_header, 'fill': fill_result_orange, 'alignment': align_center})
    ws.merge_cells('B29:F30')
    apply_styles_to_range(ws, 'B29', {
                        'font': font_final_result, 'fill': fill_result_orange, 'alignment': align_center})
    
    # --- 【新功能】新增頁尾作者標示 ---
    ws.merge_cells('A32:F32')
    ws['A32'] = "本計算機由 陳定康 設計"
    apply_styles_to_range(ws, 'A32', {'font': font_footer, 'alignment': align_right})

    # --- 備註與作者設定 ---
    comments = {
        'B2': '請輸入民國年日期。\n格式為 YYYMMDD 或 YYMMDD。\n範例：114年10月1日，請輸入 1141001。',
        'B3': '請輸入民國年日期。\n格式為 YYYMMDD 或 YYMMDD。\n範例：114年10月31日，請輸入 1141031。',
        'B4': '此選項將影響交屋當日費用歸屬：\n● 否 (預設)：交屋日歸【賣方】負責。\n● 是：交屋日歸【買方】負責。',
        'C7': '請輸入分數，例如：1/4。\n若此欄位與左側百分比欄位皆有數值，將【優先採用此分數】進行計算。',
        'B12': '若您已知曉年度總稅額，可直接填寫於此。\n系統將【優先採用此處稅額】，並忽略上方「模式一」的詳細資料。',
        'B13': '若您已知曉年度總稅額，可直接填寫於此。\n系統將【優先採用此處稅額】，並忽略上方「模式一」的詳細資料。',
        'B22': '請輸入費用已預繳之「截止日期」。\n格式為 YYYMMDD 或 YYMMDD。\n範例：預繳至114年10月31日，請輸入 1141031。',
        'B23': '請輸入費用已預繳之「截止日期」。\n格式為 YYYMMDD 或 YYMMDD。\n範例：預繳至114年10月31日，請輸入 1141031。'
    }
    for cell_ref, comment_text in comments.items():
        ws[cell_ref].comment = Comment(comment_text, "陳定康")

    # 備註區塊佈局與樣式
    ws.merge_cells('I1:N1')
    ws['I1'] = "【稅務資訊備註】"
    apply_styles_to_range(ws, 'I1', {'font': font_body_bold})
    ws.merge_cells('I2:N2')
    apply_styles_to_range(ws, 'I2', {'font': font_body_bold})
    ws.merge_cells('I3:N7')
    ws.merge_cells('I8:N8')
    apply_styles_to_range(ws, 'I8', {'font': font_body_bold})
    ws.merge_cells('I9:N13')
    apply_styles_to_range(
        ws, 'I3:I13', {'font': font_body, 'alignment': align_left, 'wrap_text': True})

    # 統一套用框線
    for r in ['A2:F4', 'A5:F10', 'A12:F13', 'A14:F18', 'A19:F25', 'A26:F30', 'I1:N13']:
        apply_styles_to_range(ws, r, {'border': thin_border})
    apply_styles_to_range(ws, 'A29:F30', {'border': medium_border})

    # --- 資料驗證 ---
    dv_land = DataValidation(type="list", formula1="=稅率表!$D$2:$D$3")
    dv_land.add('B8')
    ws.add_data_validation(dv_land)
    dv_house = DataValidation(type="list", formula1="=稅率表!$A$2:$A$14")
    dv_house.add('B10')
    ws.add_data_validation(dv_house)
    
    dv_date = DataValidation(type="custom", formula1="=AA2>=AA1")
    dv_date.errorTitle = "日期輸入錯誤"
    dv_date.error = "【房屋點交日期】必須晚於或等於【權狀登記日期】。\n\n請重新輸入正確的日期。"
    dv_date.add('B3')
    ws.add_data_validation(dv_date)

    # --- 建立稅率表工作表 ---
    ws2 = wb.create_sheet("稅率表")
    house_tax_data = [("房屋使用情境選項", "對應稅率"), ("自住用-全國單一自住(現值一定金額下)", 0.01), ("自住用-全國3戶內", 0.012), ("公益出租人/社會住宅", 0.012), ("非自住-出租(達租金標準)/繼承共有-4戶內", 0.015), ("非自住-出租(達租金標準)/繼承共有-5~6戶", 0.02),
                      ("非自住-出租(達租金標準)/繼承共有-7戶以上", 0.024), ("非自住-其他住家用-2戶內", 0.032), ("非自住-其他住家用-3~4戶", 0.038), ("非自住-其他住家用-5~6戶", 0.042), ("非自住-其他住家用-7戶以上", 0.048), ("營業用", 0.03), ("私人醫院/診所/事務所用", 0.03), ("非住家非營業用(人民團體等)", 0.02)]
    land_tax_data = [("地價稅率選項", "對應稅率"), ("自用住宅 (千分之二)",
                                          0.002), ("一般用地 (千分之十)", 0.010)]
    for r_idx, row_data in enumerate(house_tax_data, 1):
        for c_idx, cell_data in enumerate(row_data, 1):
            ws2.cell(row=r_idx, column=c_idx, value=cell_data)
    for r_idx, row_data in enumerate(land_tax_data, 1):
        for c_idx, cell_data in enumerate(row_data, 1):
            ws2.cell(row=r_idx, column=c_idx + 3, value=cell_data)
    apply_styles_to_range(ws2, 'A1:B1', {'font': font_body_bold})
    apply_styles_to_range(ws2, 'D1:E1', {'font': font_body_bold})
    apply_styles_to_range(ws2, 'B2:B14', {'number_format': '0.00%'})
    apply_styles_to_range(ws2, 'E2:E3', {'number_format': '0.000%'})
    ws2.column_dimensions['A'].width = 45
    ws2.column_dimensions['B'].width = 15
    ws2.column_dimensions['D'].width = 30
    ws2.column_dimensions['E'].width = 15
    
    # 啟用工作表保護
    ws.protection.sheet = True
    
    # --- 儲存檔案 ---
    try:
        wb.calculation.fullCalcOnLoad = True
        wb.save(filename)
        print(f"✅ 成功創建 Excel 檔案： '{filename}'")
        print("-" * 50)
        print("💡 作者資訊提示：")
        print("   - 您可以從 Excel 的 [檔案] > [資訊] 中查看作者屬性。")
        print("   - 工作表左下角已新增設計者標示。")
        print("-" * 50)

    except Exception as e:
        print(f"❌ 創建檔案時發生錯誤：{e}")


if __name__ == '__main__':
    create_final_workbook_v5_7()

✅ 成功創建 Excel 檔案： '交屋稅費計算機.xlsx'
--------------------------------------------------
💡 作者資訊提示：
   - 您可以從 Excel 的 [檔案] > [資訊] 中查看作者屬性。
   - 工作表左下角已新增設計者標示。
--------------------------------------------------


vba

In [ ]:

' 強制宣告所有變數
Option Explicit

'
' ClearInputs_Final 巨集
'
' 功能：   根據使用者提供的精確儲存格清單，清除「專業稅費分算計算機」的輸入欄位。
' 版本：   最終版
' 核心：   1. 使用使用者確認的最終儲存格清單。
'          2. 改用 .Value = "" 方法以避免 '1004' 錯誤，此方法更為穩健。
'
Public Sub ClearInputs_Final()

    ' --- 宣告變數 ---
    Dim ws As Worksheet
    Dim targetSheetName As String
    Dim cellsToClear As Variant
    Dim cellAddress As Variant
    
    ' 將工作表名稱定義為變數
    targetSheetName = "專業稅費分算計算機"
    
    ' --- 【使用者確認的最終清單】 ---
    ' 將所有要清除的儲存格地址放入一個陣列中，方便管理與修改。
    cellsToClear = Array( _
        "B2", "B3", _
        "B5", "B6", "B7", "C7", "B9", _
        "B12", "B13", _
        "B20", "B21", "B22", "B23", "B24" _
    )
    
    ' --- 錯誤處理機制 ---
    On Error GoTo ErrorHandler

    ' --- 初始化物件 ---
    Set ws = ThisWorkbook.Sheets(targetSheetName)
    
    ' --- 主程式邏輯 ---
    Application.ScreenUpdating = False
    
    ' 遍歷陣列中的每一個儲存格地址
    For Each cellAddress In cellsToClear
        ' 使用 .Value = "" 來清除內容，這是最穩定的方法
        ws.Range(cellAddress).Value = ""
    Next cellAddress
    
    ' --- 收尾工作 ---
    ' 將游標移回第一個輸入欄位
    ws.Range("B2").Select
    
    Application.ScreenUpdating = True
    
    MsgBox "指定欄位已成功清除！", vbInformation, "操作完成"
    
    Exit Sub

' --- 錯誤處理區塊 ---
ErrorHandler:
    Application.ScreenUpdating = True
    
    If Err.Number = 9 Then
        MsgBox "執行失敗！" & vbCrLf & vbCrLf & "錯誤原因：找不到名為「" & targetSheetName & "」的工作表。", vbCritical, "錯誤"
    Else
        MsgBox "執行失敗，發生未預期的錯誤！" & vbCrLf & vbCrLf & "錯誤代碼：" & Err.Number & vbCrLf & "錯誤描述：" & Err.Description, vbCritical, "錯誤"
    End If
    
    Err.Clear

End Sub


'\n\' 強制宣告所有變數\nOption Explicit\n\n\'\n\' ClearInputs_Final 巨集\n\'\n\' 功能：   根據使用者提供的精確儲存格清單，清除「專業稅費分算計算機」的輸入欄位。\n\' 版本：   最終版\n\' 核心：   1. 使用使用者確認的最終儲存格清單。\n\'          2. 改用 .Value = "" 方法以避免 \'1004\' 錯誤，此方法更為穩健。\n\'\nPublic Sub ClearInputs_Final()\n\n    \' --- 宣告變數 ---\n    Dim ws As Worksheet\n    Dim targetSheetName As String\n    Dim cellsToClear As Variant\n    Dim cellAddress As Variant\n    \n    \' 將工作表名稱定義為變數\n    targetSheetName = "專業稅費分算計算機"\n    \n    \' --- 【使用者確認的最終清單】 ---\n    \' 將所有要清除的儲存格地址放入一個陣列中，方便管理與修改。\n    cellsToClear = Array( _\n        "B2", "B3", _\n        "B5", "B6", "B7", "C7", "B9", _\n        "B12", "B13", _\n        "B20", "B21", "B22", "B23", "B24" _\n    )\n    \n    \' --- 錯誤處理機制 ---\n    On Error GoTo ErrorHandler\n\n    \' --- 初始化物件 ---\n    Set ws = ThisWorkbook.Sheets(targetSheetName)\n    \n    \' --- 主程式邏輯 ---\n    Application.ScreenUpdating = False\n    \n    \' 遍歷陣列中的每一個儲存格地址\n    For Each cellAddress In cellsToClear\n        \' 使用 .Value =

In [ ]:
' VBA 程式碼開始
Sub ExportWholeSheetAsImage()
    ' --- 參數設定 ---
    Dim ws As Worksheet
    Dim rng As Range
    Dim tempChart As ChartObject
    Dim imagePath As String

    ' 設定要操作的工作表名稱
    Set ws = ThisWorkbook.Sheets("專業稅費分算計算機")

    ' 自動偵測工作表已使用的全部範圍
    Set rng = ws.UsedRange

    ' 設定圖片儲存的路徑與檔名 (會存在與 Excel 檔相同的位置)
    imagePath = ThisWorkbook.Path & "\\稅費分算總表.png"

    ' --- 核心執行區 ---
    ' 複製指定範圍為圖片
    rng.CopyPicture Appearance:=xlScreen, Format:=xlPicture

    ' 建立一個暫時的圖表物件來容納圖片
    Set tempChart = ws.ChartObjects.Add(Left:=0, Top:=0, Width:=rng.Width, Height:=rng.Height)

    ' 將複製的圖片貼到圖表中
    With tempChart.Chart
        .Paste
        ' 將圖片匯出成 PNG 檔案
        .Export Filename:=imagePath, FilterName:="PNG"
    End With

    ' 刪除暫時的圖表
    tempChart.Delete

    ' --- 完成提示 ---
    MsgBox "✅ 完整的稅費分算總表圖片已成功匯出！" & vbNewLine & "檔案位置：" & imagePath, vbInformation, "匯出成功"

End Sub
' VBA 程式碼結束